In [2]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

/kaggle/input/sentiment-analysis-for-mental-health/Combined Data.csv


In [3]:
import numpy as np
import pandas as pd

In [4]:
data = pd.read_csv('/kaggle/input/sentiment-analysis-for-mental-health/Combined Data.csv')

In [5]:
data.shape

(53043, 3)

In [6]:
data.columns

Index(['Unnamed: 0', 'statement', 'status'], dtype='object')

In [7]:
data['status'].unique()


array(['Anxiety', 'Normal', 'Depression', 'Suicidal', 'Stress', 'Bipolar',
       'Personality disorder'], dtype=object)

In [8]:
data = data.drop('Unnamed: 0',axis=1)

In [9]:
data.shape

(53043, 2)

In [10]:
data.isna().sum()

statement    362
status         0
dtype: int64

In [11]:
data.head(3)

,statement,status
0,oh my gosh,Anxiety
1,"trouble sleeping, confused mind, restless hear...",Anxiety
2,"All wrong, back off dear, forward doubt. Stay ...",Anxiety


In [12]:
data['status'].value_counts()



status
Normal                  16351
Depression              15404
Suicidal                10653
Anxiety                  3888
Bipolar                  2877
Stress                   2669
Personality disorder     1201
Name: count, dtype: int64

In [13]:
data.shape

(53043, 2)

In [14]:
p_status = data['status'].tolist()
p_statement = data['statement'].tolist()

From the above data the mapping is 0 -> Positive and 1 -> Negative

In [15]:
sample_statement = p_statement[0]
sample_status = p_status[0]

In [16]:
sample_statement,sample_status

('oh my gosh', 'Anxiety')

## Module1 : Classification

In [17]:
from transformers import pipeline

classifier = pipeline("zero-shot-classification",
                     model="facebook/bart-large-mnli")

2025-12-10 19:38:39.293429: E external/local_xla/xla/stream_executor/cuda/cuda_fft.cc:477] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
E0000 00:00:1765395519.463252      47 cuda_dnn.cc:8310] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
E0000 00:00:1765395519.511474      47 cuda_blas.cc:1418] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered


AttributeError: 'MessageFactory' object has no attribute 'GetPrototype'

AttributeError: 'MessageFactory' object has no attribute 'GetPrototype'

AttributeError: 'MessageFactory' object has no attribute 'GetPrototype'

AttributeError: 'MessageFactory' object has no attribute 'GetPrototype'

...

tokenizer_config.json:   0%|          | 0.00/26.0 [00:00<?, ?B/s]

vocab.json: 0.00B [00:00, ?B/s]

merges.txt: 0.00B [00:00, ?B/s]

tokenizer.json: 0.00B [00:00, ?B/s]

Device set to use cuda:0


In [23]:
categories = data['status'].unique().tolist()


In [24]:

result = classifier(sample_statement, categories)

print(f"Statement: {sample_statement}\n")
print(f"Predicted: {result['labels'][0]}")
print(f"Confidence: {result['scores'][0]:.3f}")
print(f"Actual: {sample_status}\n")

Statement: oh my gosh

Predicted: Stress
Confidence: 0.385
Actual: Anxiety



In [26]:
# Check statement lengths
data['statement_length'] = data['statement'].str.split().str.len()

print("Statement length statistics:")
print(data['statement_length'].describe())

print("\nShortest statements:")
print(data.nsmallest(5, 'statement_length')[['statement', 'status']])

print("\nLongest statements:")
print(data.nlargest(5, 'statement_length')[['statement', 'status']])

Statement length statistics:
count    52681.000000
mean       113.158729
std        163.735305
min          1.000000
25%         15.000000
50%         62.000000
75%        148.000000
max       6300.000000
Name: statement_length, dtype: float64

Shortest statements:
      statement   status
88      Anxious  Anxiety
424     anxious  Anxiety
753       chat.   Normal
834      cement   Normal
839  SEBASTIANN   Normal

Longest statements:
                                               statement                status
13293  I cannot TAKE IT ANYMORE. I cannot TAKE IT ANY...              Suicidal
51396  Please help me understand what I went through ...  Personality disorder
10834  I have only 1 person I can somewhat open to bu...              Suicidal
46660  DEPRESSION HAS A PURPOSE: HOW TO USE IT RIGHT ...               Bipolar
39752  it doesn t matter anymore i m going to copy an...            Depression


In [33]:
data['statement'].isna().sum()

362

In [41]:
data_clean = data[data['statement'].notna()]
p_statements = data_clean['statement'].tolist()
p_status = data_clean['status'].tolist()
p_status_sample = p_status[:100] 

In [36]:
sample_size = 100
predicted_labels = []

for i, statement in enumerate(p_statements[:sample_size]):
    result = classifier(statement, categories)
    predicted_labels.append(result['labels'][0])
    
    if (i + 1) % 20 == 0:
        print(f"Processed {i + 1}/{sample_size}...")

print(f"\n Finished! Processed {len(predicted_labels)} statements")

Processed 20/100...
Processed 40/100...
Processed 60/100...
Processed 80/100...
Processed 100/100...

 Finished! Processed 100 statements


In [42]:
from sklearn.metrics import accuracy_score, f1_score, classification_report
accuracy = accuracy_score(p_status_sample, predicted_labels)
f1 = f1_score(p_status_sample, predicted_labels, average='weighted')
print(f"Accuracy: {accuracy:.2%}")
print(f"F1 Score (weighted): {f1:.3f}")
print("\n Classification Report:")
print(classification_report(p_status_sample, predicted_labels))

Accuracy: 31.00%
F1 Score (weighted): 0.473

 Classification Report:
              precision    recall  f1-score   support

     Anxiety       1.00      0.31      0.47       100
     Bipolar       0.00      0.00      0.00         0
  Depression       0.00      0.00      0.00         0
      Normal       0.00      0.00      0.00         0
      Stress       0.00      0.00      0.00         0

    accuracy                           0.31       100
   macro avg       0.20      0.06      0.09       100
weighted avg       1.00      0.31      0.47       100



/usr/local/lib/python3.11/dist-packages/sklearn/metrics/_classification.py:1344: UndefinedMetricWarning: Recall and F-score are ill-defined and being set to 0.0 in labels with no true samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
/usr/local/lib/python3.11/dist-packages/sklearn/metrics/_classification.py:1344: UndefinedMetricWarning: Recall and F-score are ill-defined and being set to 0.0 in labels with no true samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
/usr/local/lib/python3.11/dist-packages/sklearn/metrics/_classification.py:1344: UndefinedMetricWarning: Recall and F-score are ill-defined and being set to 0.0 in labels with no true samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))


## Module 2 : Summarization

In [43]:
summarizer = pipeline("summarization", model="facebook/bart-large-cnn")


config.json: 0.00B [00:00, ?B/s]

model.safetensors:   0%|          | 0.00/1.63G [00:00<?, ?B/s]

generation_config.json:   0%|          | 0.00/363 [00:00<?, ?B/s]

vocab.json: 0.00B [00:00, ?B/s]

merges.txt: 0.00B [00:00, ?B/s]

tokenizer.json: 0.00B [00:00, ?B/s]

Device set to use cuda:0


In [62]:
long_statements = data_clean[data_clean['statement_length'] > 200]

long_statements.count()

statement           9029
status              9029
statement_length    9029
dtype: int64

In [63]:
long_s = long_statements['statement'].tolist()

In [64]:
sample_text = long_s[0]
sample_text

'I do not know how to navigate these feelings, not that its a new feeling by any stretch. I just do not understand how I can go on from being so pleasantly fine to suicidal and self loathing within in minutes and not be able to step out of it. I have been suffering from poor mental health for over 10 years and I am about to turn 26 next week and I feel so ashamed for how serious my negative thoughts are. Covid was a hard transition especially having moved to a new city a few months before everything started locking down, but now I am here and settled and I have a loving partner, a great connection to their family, a really fun job and I finished my degree. But when I stop and look around I feel like nothing will ever be good enough for me and I will never be good enough, I will always have skin problems and bad teeth and acne. I will never be the artist I used to be (I have a Bachelor in fine art) because I do not have the same drive or passionate that I used to have for it. I will nev

In [66]:
summary = summarizer(sample_text, max_length=70, min_length=20, do_sample=False)
print(f"Summary ({len(summary[0]['summary_text'].split())} words):")
print(summary[0]['summary_text'])
 

Summary (63 words):
"I have been suffering from poor mental health for over 10 years and I am about to turn 26 next week and I feel so ashamed for how serious my negative thoughts are" "I just do not understand how I can go on from being so pleasantly fine to suicidal and self loathing within in minutes and not be able to step out
